In [1]:
API_KEY = "PER4V6RYCAU4TZ54M69D9GXHWNVKX6DY7Z"
BASE_URL = "https://api.etherscan.io/api"
ETHER_VALUE = 10 ** 18
address = "0x73bceb1cd57c711feac4224d062b0f6ff338501e"
address2 = "0xddbd2b932c763ba5b1b7ae3b362eac3e8d40121a"

In [2]:
len("0x73bceb1cd57c711feac4224d062b0f6ff338501e")

42

In [3]:
import pandas as pd
from requests import get
from matplotlib import pyplot as plt
from datetime import datetime

import matplotlib.ticker as mticker
from matplotlib.gridspec import GridSpec # 畫圖網格佈局

In [4]:
test_url_2 = "https://etherscan.io/tx/0xe06bfabbacc926d7bbcb6a6ab0ab9df5026c7f53d7722874148c42f65ca25878"

In [5]:
responce2 = get(test_url_2)
print(responce2)

<Response [403]>


In [6]:
test_url = "https://api.etherscan.io/api?module=contract&action=getsourcecode&address=0x7a250d5630b4cf539739df2c5dacb4c659f2488d&apikey=PER4V6RYCAU4TZ54M69D9GXHWNVKX6DY7Z"


In [7]:
responce = get(test_url).json()

In [8]:
responce["result"][0].keys()

dict_keys(['SourceCode', 'ABI', 'ContractName', 'CompilerVersion', 'OptimizationUsed', 'Runs', 'ConstructorArguments', 'EVMVersion', 'Library', 'LicenseType', 'Proxy', 'Implementation', 'SwarmSource'])

In [9]:
responce["result"][0]["ContractName"]

'UniswapV2Router02'

### Build up class

In [79]:
class Get_address_details():
    
    def __init__(self, address):
       
        self.address = address
        self.BASE_URL = "https://api.etherscan.io/api"
        self.ETHER_VALUE = 10 ** 18
        self.API_KEY = "PER4V6RYCAU4TZ54M69D9GXHWNVKX6DY7Z"
        
    def make_api_url(self, module, action, **kwargs):
        url = self.BASE_URL + f"?module={module}&action={action}&address={self.address}&apikey={self.API_KEY}"

        for key, value in kwargs.items():
            url += f"&{key}={value}"

        return url
    
    def make_hash_api_url(self, module, action, txhash, **kwargs):
        url = self.BASE_URL + f"?module={module}&action={action}&txhash={txhash}&apikey={self.API_KEY}"
        
        for key, value in kwargs.items():
            url += f"&{key}={value}"

        return url
    
    def get_txns_from_txnhash(self, txhash):
        txn_url = self.make_hash_api_url("account", "txlistinternal", txhash)
        responce = get(txn_url)
        data = responce.json()
        
        return data["result"]
        
    def get_account_balance(self):
        balance_url = self.make_api_url("account", "balance", tag="latest")
        response = get(balance_url)
        data = response.json()

        value = int(data["result"]) / ETHER_VALUE
        return value
    
    def get_all_txns(self):
        """
        Get all txns 
        """
        transactions_url = self.make_api_url("account", "txlist", startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
        response = get(transactions_url)
        nor_data = response.json()["result"]
        nor_df = pd.DataFrame(nor_data)
        nor_df.insert(1, "txn_type", value ="normal")
        
        internal_tx_url = self.make_api_url("account", "txlistinternal", startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
        in_response = get(internal_tx_url)
        in_data = in_response.json()["result"]
        in_df = pd.DataFrame(in_data)
        in_df.insert(1, "txn_type", value = "internal")
        
        data_df = pd.concat([nor_df, in_df], join = "outer")
        
        new_time_list = []
        for idx, timestamp in enumerate(data_df["timeStamp"]):
            data_df["timeStamp"][idx] = int(timestamp)
            new_time = str(datetime.fromtimestamp(int(timestamp))).split(" ")[0]
            new_time_list.append(new_time)
        data_df.insert(1, "datetime", new_time_list)
        data_df["datetime"] = pd.to_datetime(data_df["datetime"], format="%Y-%m-%d")
        date_df_out = data_df.set_index("datetime")
        # data_df.reset_index(inplace=True, drop=True)
        
        return date_df_out
    

In [305]:
def get_txn_counts(address_df, txn_type, txn_time):

    if txn_type == "all" and txn_time == "all":
        year_list = {str(i):len(address_df.loc[str(i)]) for i in range(2015, 2023)}
        result = pd.DataFrame(list(year_list.items()), columns = ["year", "counts"])
        return result
    
    elif txn_type == "all" and txn_time != "all":
        year_list_output = {str(month):0 for month in range(1, 13)}
        month_count = {str(i):len(address_df.loc["{}-{}".format(txn_time, i)]) for i in range(1, 13)}
        year_list_output.update(month_count)
        result = pd.DataFrame(list(year_list_output.items()), columns = ["month", "counts"])
        return result
    
    elif txn_type != "all" and txn_time == "all":
        mask = address_df["txn_type"] == txn_type
        year_list = {str(i):len(address_df[mask].loc[str(i)]) for i in range(2015, 2023)}
        result = pd.DataFrame(list(year_list.items()), columns = ["year", "counts"])
        return result

    else:            
        year_list_output = {str(month):0 for month in range(1, 13)}

        for i in range(1, 13):
            tmp_df = address_df.loc["{}-{}".format(txn_time, i)]
            tmp_mask = tmp_df["txn_type"] == txn_type
            year_list_output.update(len(tmp_df[tmp_mask]))

        result = pd.DataFrame(list(year_list_output.items()), columns = ["month", "counts"])
        return result

In [306]:
get_txns = Get_address_details("0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045")

In [307]:
pd.set_option("display.max_columns", None)
all_txn_result = get_txns.get_all_txns()

In [308]:
txn_type = "all"
txn_time = "2022"
mask = all_txn_result["txn_type"] == txn_type


In [309]:
get_txn_counts(all_txn_result, txn_type, txn_time)

,month,counts
0,1,48
1,2,40
2,3,28
3,4,27
4,5,57
5,6,28
6,7,16
7,8,0
8,9,0
9,10,0


In [312]:
test = all_txn_result.loc["2022-8"]
test

,blockNumber,txn_type,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,gasPrice,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations,methodId,functionName,type,traceId,errCode
datetime,,,,,,,,,,,,,,,,,,,,,,,,


In [311]:
test_txn_type = "normal"
test_mask = test["txn_type"] == test_txn_type
test[test_mask]

,blockNumber,txn_type,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,gasPrice,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations,methodId,functionName,type,traceId,errCode
datetime,,,,,,,,,,,,,,,,,,,,,,,,


In [291]:
get_txn_counts(all_txn_result, txn_type, txn_time)

KeyError: '2022-8'

In [ ]:
txn_type = "normal"
txn_time = "2020"
# mask = all_txn_result["txn_type"] == txn_type
# year_list = {"{}".format(i):len(all_txn_result[mask]["{}-{}".format(txn_time, i)]) for i in range(1, 13)}
# year_list = {"{}".format(i):len(all_txn_result["{}-{}".format(txn_time, i)]) for i in range(1, 13)}

mask = all_txn_result["txn_type"] == txn_type
year_list = {"{}".format(i):len(all_txn_result[mask]["{}".format(i)]) for i in range(2015, 2023)}
result = pd.DataFrame(list(year_list.items()), columns = ["year", "counts"])

In [ ]:
year_list

In [ ]:
year_list = {"{}".format(i):len(all_txn_result["{}".format(i)]) for i in range(2015, 2023)}

In [ ]:
year_list

In [ ]:
pd.DataFrame(list(year_list.items()),
                   columns=['year', 'counts'])

In [ ]:
pd.DataFrame(year_list, columns = ["datetime", "counts"])

In [ ]:
time_mask = (lambda x : datetime.strptime(x, "2020-%m-%d"))

In [ ]:
all_txn_result[all_txn_result["datetime"].apply(lambda x : datetime.strptime(x, "2020-%m-%d"))]

In [ ]:
new_all_txn_result = all_txn_result.set_index("datetime")

In [ ]:
new_all_txn_result[""]

In [ ]:
get_txns.get_normal_txns()

In [ ]:
for txn in get_txns.get_internal_txns():
    print("From :", txn["from"])
    print("To :", txn["to"])
    print("Value :", int(txn["value"])/ETHER_VALUE)
    print("Type :", txn["type"])
    print("Hash :", txn["hash"])
    print("="*10)

In [ ]:
get_txns.get_all_txns()

In [ ]:
get_txns = Get_address_details(address)
result = get_txns.get_all_txns_by_datetime()
result

In [ ]:
def txns_by_year(address_result_dict):
    datetime_list = list(address_result_dict.keys())
    yeartime_dict = {}
    
    for year_day in address_result_dict:
        year = year_day.split("-")[0]
        if year in yeartime_dict:
            yeartime_dict[year] += 1
        else:
            yeartime_dict[year] = 1
    
    return yeartime_dict
        

In [ ]:
txns_by_year(result)

In [ ]:
def txn_in_specific_year(target_year, address_result_dict):
    
    month_time_dict = {}
    
    for year_day in address_result_dict:
        year, month = year_day.split("-")[0], year_day.split("-")[1]
        if year == target_year:
            if month in month_time_dict:
                month_time_dict[month] += 1
            else:
                month_time_dict[month] = 1
        else:
            pass
        
    return month_time_dict

In [ ]:
txn_in_specific_year("2019", result)

In [ ]:
df = pd.DataFrame(list(result.items()),
                   columns=['datetime', 'counts'])

In [ ]:
df

In [ ]:
list(df.items())[0][1].tolist()

In [ ]:
list(df.items())[1][1].tolist()

In [ ]:
len(df)

In [ ]:
# example for get account balance url
"""
https://api.etherscan.io/api
   ?module=account
   &action=balance
   &address=0xde0b295669a9fd93d5f28d9ec85e40f4cb697bae
   &tag=latest
   &apikey=YourApiKeyToken
"""

### Create api url

In [ ]:
def make_api_url(module, action, address, **kwargs):
    url = BASE_URL + f"?module={module}&action={action}&address={address}&apikey={API_KEY}"

    for key, value in kwargs.items():
        url += f"&{key}={value}"

    return url

### Get account balance by address

In [ ]:
def get_account_balance(address):
    balance_url = make_api_url("account", "balance", address, tag="latest")
    response = get(balance_url)
    data = response.json()

    value = int(data["result"]) / ETHER_VALUE
    return value

In [ ]:
account_balance = get_account_balance(address)
print(account_balance)

In [ ]:
# Example for get all txns record from address
"""
https://api.etherscan.io/api
   ?module=account
   &action=txlist
   &address=0xc5102fE9359FD9a28f877a67E36B0F050d81a3CC
   &startblock=0
   &endblock=99999999
   &page=1
   &offset=10
   &sort=asc
   &apikey=YourApiKeyToken
"""

In [ ]:
def get_txns(address):
    """
    Get all txns 
    """
    transactions_url = make_api_url("account", "txlist", address, startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
    response = get(transactions_url)
    data = response.json()["result"]

    internal_tx_url = make_api_url("account", "txlistinternal", address, startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
    response2 = get(internal_tx_url)
    data2 = response2.json()["result"]

    data.extend(data2)
    data.sort(key=lambda x: int(x['timeStamp']))
    
    return data

In [ ]:
def get_txns_sum_by_datetime(address):
    address_data = get_txns(address)
    txns_by_datetime = {}
    
    for txn_data in address_data:
        time = str(datetime.fromtimestamp(int(txn_data["timeStamp"]))).split(" ")[0]
        
        if time in txns_by_datetime:
            txns_by_datetime[time] += 1
        else:
            txns_by_datetime[time] = 1
    return txns_by_datetime

In [ ]:
def get_normal_txns_by_datetime(address):
    
    """
    Get normal txns by date time
    """
    
    txns_by_datetime = {}
    
    transactions_url = make_api_url("account", "txlist", address, startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
    response = get(transactions_url)
    data = response.json()["result"]
    
    data.sort(key=lambda x: int(x['timeStamp']))
    
    for d in data:
        time = str(datetime.fromtimestamp(int(d["timeStamp"]))).split(" ")[0]
        
        if time in txns_by_datetime:
            txns_by_datetime[time] += 1
        else:
            txns_by_datetime[time] = 1
        
    return txns_by_datetime
    

In [ ]:
noraml_txns = get_normal_txns_by_datetime(address)

In [ ]:
noraml_txns

In [ ]:
def get_internal_txns_by_datetime(address):
    
    """
    Get normal txns by date time
    """
    
    txns_by_datetime = {}
    
    transactions_url = make_api_url("account", "txlistinternal", address, startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
    response = get(transactions_url)
    data = response.json()["result"]
    
    data.sort(key=lambda x: int(x['timeStamp']))
    
    for d in data:
        time = str(datetime.fromtimestamp(int(d["timeStamp"]))).split(" ")[0]
        
        if time in txns_by_datetime:
            txns_by_datetime[time] += 1
        else:
            txns_by_datetime[time] = 1
        
    return txns_by_datetime

In [ ]:
internal_txns_datetime = get_internal_txns_by_datetime(address2)

In [ ]:
internal_txns_datetime

In [ ]:
tick_spacing = len(internal_txns_datetime)/7 # x軸密集度

fig = plt.figure(figsize=(10, 8)) # 寬10 長8
gs = GridSpec(1, 1) # 設立1x1的網格

ax = plt.subplot(gs[0, 0]) # 子圖ax在網格的[0, 0]
ax.xaxis.set_major_locator(mticker.MultipleLocator(tick_spacing))

ax.plot_date(list(internal_txns_datetime.keys()), list(internal_txns_datetime.values()), linestyle ='solid')

### Ploting txns (nomal) sum by datetime

In [ ]:
normal_txns_datetime = get_normal_txns_by_datetime(address2)

In [ ]:
tick_spacing = len(normal_txns_datetime)/7 # x軸密集度

fig = plt.figure(figsize=(10, 8)) # 寬10 長8
gs = GridSpec(1, 1) # 設立1x1的網格

ax = plt.subplot(gs[0, 0]) # 子圖ax在網格的[0, 0]
ax.xaxis.set_major_locator(mticker.MultipleLocator(tick_spacing))

ax.plot_date(list(normal_txns_datetime.keys()), list(normal_txns_datetime.values()), linestyle ='solid')

### Ploting txns (nomal and internal) sum by datetime

In [ ]:
txns_datetime = get_txns_sum_by_datetime(address2)

In [ ]:
tick_spacing = len(txns_datetime)/7 # x軸密集度

fig = plt.figure(figsize=(10, 8)) # 寬10 長8
gs = GridSpec(1, 1) # 設立1x1的網格

ax = plt.subplot(gs[0, 0]) # 子圖ax在網格的[0, 0]
ax.xaxis.set_major_locator(mticker.MultipleLocator(tick_spacing))

ax.plot_date(list(txns_datetime.keys()), list(txns_datetime.values()), linestyle ='solid')